In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator

from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [4]:
%cd drive/MyDrive

/content/drive/MyDrive


In [5]:
!unzip archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_116.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_118.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_126.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_134.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_141.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_168.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_175.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_183.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_221.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_222.png  
  inflating: cell_i

In [6]:
!ls

 archive.zip  'Colab Notebooks'		 intestine   malaria
 cell_images   IMG-20240216-WA0048.jpg	 liver


In [8]:
%cd cell_images
!ls


/content/drive/MyDrive/cell_images
cell_images  Parasitized  Uninfected


In [9]:
!ls

cell_images  Parasitized  Uninfected


In [10]:
for i in os.listdir("cell_images"):
  print(i)

Parasitized
Uninfected


In [11]:
os.listdir("cell_images")

['Parasitized', 'Uninfected']

In [12]:
!ls cell_images/

Parasitized  Uninfected


In [13]:
for i in os.listdir("cell_images/"):
 print(i,len(os.listdir("cell_images/"+i)))

Parasitized 13780
Uninfected 13780


In [14]:
try:
 os.mkdir("train")
 os.mkdir("test")
except:
  pass
for i in os.listdir("cell_images/"):
  try:
   os.mkdir("train/"+i)
   os.mkdir("test/"+i)
  except:
   pass
  for j in os.listdir("cell_images/"+i)[:9000]:
    os.rename("cell_images/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("cell_images/"+i)[:4780]:
    os.rename("cell_images/"+i+"/"+j,"test/"+i+"/"+j)

In [15]:
!ls test

Parasitized  Uninfected


In [16]:
def img_Data(dir_path,target_size,batch,class_1st,preporcssing):
  if preporcssing:
    gen_object =ImageDataGenerator(preprocessing_function=preporcssing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode='sparse',
                                        classes=class_1st,
                                        shuffle=True))

In [17]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 18000 images belonging to 2 classes.
Found 9558 images belonging to 2 classes.


In [18]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax')

9406464/9406464 [==============================] - 0s 0us/step


In [19]:
base_model.trainable = False

In [20]:
from keras.src.layers.serialization import activation
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint('.md2_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3579978 (13.66 MB)
Trainable params: 1321994 (5.04 MB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [23]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 0.5604 - accuracy: 0.7925

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 192s 5s/step - loss: 0.5604 - accuracy: 0.7925 - val_loss: 0.2626 - val_accuracy: 0.8959
Epoch 2/10
36/36 [==============================] - 151s 4s/step - loss: 0.1808 - accuracy: 0.9346 - val_loss: 0.2228 - val_accuracy: 0.9228
Epoch 3/10
36/36 [==============================] - 149s 4s/step - loss: 0.1568 - accuracy: 0.9440 - val_loss: 0.2109 - val_accuracy: 0.9272
Epoch 4/10
36/36 [==============================] - 150s 4s/step - loss: 0.1483 - accuracy: 0.9462 - val_loss: 0.2028 - val_accuracy: 0.9291
Epoch 5/10
36/36 [==============================] - 147s 4s/step - loss: 0.1377 - accuracy: 0.9508 - val_loss: 0.1926 - val_accuracy: 0.9314
Epoch 6/10
36/36 [==============================] - 148s 4s/step - loss: 0.1330 - accuracy: 0.9521 - val_loss: 0.1938 - val_accuracy: 0.9288
Epoch 7/10
36/36 [==============================] - 148s 4s/step - loss: 0.1302 - accuracy: 0.9523 - val_loss: 0.1926 - val_accuracy: 0.9324
Epoch 8/10
36/36 [======